<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [12]:
path = Path('medical-concept-normalization/data_collection')

In [13]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [14]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [15]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [16]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [17]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [18]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [19]:
data_clas.add_test(test,label=0)


In [20]:
data_lm.save()
data_clas.save()

In [21]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'cramp',
 'weakness',
 'limb',
 'sodium',
 'by',
 'on',
 'stomach',
 '-',
 'depression',
 'back',
 'is',
 'patients',
 'cramps',
 'as',
 'joint',
 'that',
 'upper',
 'gas',
 'at',
 'or',
 'memory',
 'drug',
 'mg',
 'leg',
 'legs',
 'pains',
 'headache',
 'from',
 'knee',
 'unable',
 'shoulder',
 'group',
 'not',
 'arthralgia',
 'gastrointestinal',
 'study',
 'treatment',
 'excessive',
 'neck',
 'foot',
 'all',
 'this',
 'arthritis',
 'aches',
 'hip',
 'after',
 'numbness',
 'arthrotec',
 'release',
 'voltaren',
 'insomnia',
 'p',
 'nausea',
 '/',
 'disease',
 'bleeding',
 'increased',
 'an',
 'walk',
 'both',
 'asthenia',
 'swelling',
 'dizziness',
 'skin',
 'abdominal',
 'like',
 'heart',
 'be',
 'tired',
 'feelin

In [22]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [23]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.780315,3.386974,0.456842,1:33:10


In [25]:
# learn.fit_one_cycle(1, 1e-2)

In [26]:
# learn.fit_one_cycle(1, 1e-3)

In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.413204,3.328706,0.461384,1:32:24


In [28]:
learn.save_encoder('pubmed_ask_fold5_first')

In [29]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.967611,3.523159,0.371648,03:52


In [30]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.930011,3.013497,0.466156,04:32


In [31]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.364975,2.590929,0.527458,08:52


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.013470,2.285984,0.595147,09:58
1,1.700988,2.099511,0.629630,10:12


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.592432,1.928155,0.644955,09:21
1,1.386737,1.822559,0.680715,09:35


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.230929,1.667969,0.696041,09:06
1,1.078532,1.602040,0.706258,09:26


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.061169,1.514435,0.721584,09:38
1,0.849137,1.446217,0.734355,09:20


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.842305,1.423262,0.734355,08:49
1,0.741426,1.329082,0.758621,09:50


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.712314,1.333238,0.758621,09:07
1,0.695249,1.277151,0.763729,09:04
2,0.565292,1.240200,0.770115,09:27
3,0.475601,1.225326,0.780332,09:47


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.495066,1.242786,0.768838,09:07
1,0.515897,1.236981,0.777778,08:56
2,0.454311,1.191340,0.791826,09:28
3,0.363375,1.170614,0.798212,10:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.380325,1.155571,0.793103,09:32
1,0.422720,1.180089,0.787995,10:08
2,0.355390,1.110142,0.803321,09:56
3,0.301288,1.149689,0.803321,09:43


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.325772,1.146589,0.802043,09:27
1,0.338919,1.130369,0.803321,09:23
2,0.283533,1.171353,0.822478,09:29
3,0.269689,1.119172,0.822478,09:41


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.250872,1.113349,0.821200,09:58
1,0.292270,1.166984,0.807152,09:13
2,0.256098,1.123202,0.833972,09:12
3,0.213805,1.115637,0.836526,08:55


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.248544,1.147261,0.817369,09:23
1,0.273241,1.189617,0.817369,09:02
2,0.239500,1.150547,0.828863,09:59
3,0.203521,1.128514,0.822478,09:30


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.213866,1.132328,0.823755,10:02
1,0.245891,1.169121,0.825032,09:50
2,0.210952,1.132126,0.826309,10:01
3,0.200280,1.117414,0.827586,09:04


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.214610,1.197825,0.825032,08:53
1,0.210810,1.097320,0.826309,09:16


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.225079,1.182037,0.828863,10:06
1,0.219667,1.187382,0.833972,09:29


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.206944,1.175350,0.823755,09:33


In [47]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold5')

# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[0, 861, 2, 2, 211, 8, 8, 8, 8, 16, 16, 16, 19, 27, 644, 27, 110, 197, 197, 197, 37, 37, 37, 37, 37, 37, 38, 48, 49, 617, 49, 49, 49, 49, 53, 53, 53, 58, 62, 75, 616, 71, 71, 71, 71, 76, 209, 864, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 300, 83, 733, 206, 92, 94, 94, 94, 94, 94, 94, 95, 96, 578, 96, 98, 100, 100, 100, 100, 545, 100, 100, 100, 100, 103, 104, 287, 107, 107, 108, 108, 108, 111, 730, 111, 111, 111, 111, 111, 111, 111, 616, 119, 122, 515, 122, 122, 125, 616, 2, 125, 125, 125, 125, 125, 135, 616, 137, 137, 344, 148, 529, 151, 157, 865, 375, 509, 166, 166, 166, 171, 172, 172, 289, 172, 172, 172, 172, 172, 172, 172, 173, 173, 214, 902, 201, 165, 904, 904, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 184, 187, 187, 190, 190, 190, 190, 289, 188, 190, 190, 190, 190, 190, 190, 265, 193, 644, 197, 197, 197, 197, 197, 202, 202, 205, 209, 210, 210, 211,

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

686
0.792147806004619
